<a href="https://colab.research.google.com/github/ortizkate/ortizkate/blob/main/Test_Decoder_Pye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import struct
import base64
from base64 import binascii
import pandas as pd
import numpy as np

In [ ]:
def readUInt16LE(bytes):
    value = (bytes[1] << 8) + bytes[0]
    return (value & 0xFFFF)

In [ ]:
def read_int16_le(bytes):
    ref = read_uint16_le(bytes)
    return ref - 0x10000 if ref > 0x7fff else ref

In [ ]:
def readUInt32LE(bytes): 
    value = (bytes[3] << 24) + (bytes[2] << 16) + (bytes[1] << 8) + bytes[0]
    return (value & 0xFFFFFFFF)

In [ ]:
def readInt32LE(bytes):
    ref = readUInt32LE(bytes)
    if ref > 0x7FFFFFFF:
        return(ref - 0x100000000)
    return (ref)

In [ ]:
def readFloatLE(bytes):
    str_val=''
    for a in reversed(bytes):
        str_val = str_val+f'{a:02X}'
        #print(f'{a:02X}')
    
    #print(''.join(f'{i:02x}' for i in bytes))
    f = struct.unpack('!f', bytes.fromhex(str_val))[0]   

    # print("py:",bytes.hex())
    # print("my:",str_val)
    return f

In [ ]:
def decode_payload_em500udl(payload_str):
    payload_hex = base64.urlsafe_b64decode(payload_str)
    # print("Payload Raw:",payload_str)
    # print("Payload Hex:",payload_hex.hex())
    decode_out=[]
    k = 0
    i = 0
    while k< len(payload_hex):
        ch_id   = payload_hex[i]
        ch_type = payload_hex[i+1]
        # Battery
        if ch_id == 0x01 and ch_type == 0x75:
            ch_val  = payload_hex[i+2]
            decode_out.append(['bat', ch_val])
            i = i + 3

        # Distance
        ch_id   = payload_hex[i]
        ch_type = payload_hex[i+1]
        if ch_id == 0x03 and ch_type == 0x82:
            ch_val  = payload_hex[i+3] << 8  | payload_hex[i+2] 
            decode_out.append(['val', ch_val])
            i = i + 4
        break
        
        # POSITION
        # ch_id   = payload_hex[i]
        # ch_type = payload_hex[i+1]
        # ch_val  = payload_hex[i+2]
        # if ch_id == 0x04 and ch_type == 0x00:
        #     decode_out.append(['pos', 'tilt']) if ch_val==1 else decode_out.append(['pos', 'normal'])                
        # break

    return (decode_out)

In [ ]:
# Validando Código EM500UDL
Trama_DB='AXVkA4JyBQOCcgU='
s = decode_payload_em500udl(Trama_DB)
print(s)

[['bat', 100], ['val', 1394]]
